This GESIS notebook demonstrates the functionality of M3 to infer the gender of people using images. It is a supplement to the [GESIS Gender Inference website](http://193.175.238.89/Gender_Inference/mixed.html), which includes more details, guidelines and limitations that you should consider before using this notebook.

**Please be patient and wait until the notebook loads completely. Do not restart or refresh the page**

1. If notebook is not run on the launch, please run all cells by clicking "Cell" -> "Run all" <br/>
2. Upload several photos from your computer by clicking "Upload" <br/>
3. If you want to run M3 with both images and names select an option "With names". Filenames will be regarded as names. <br/>
4. Press "Submit" to run the model on your data. The outcome will be displayed in the table with predicted gender and confidence. The results are also saved to the csv file <br/>
4. Button "Clear" ensures no data is stored afterwards

<b>Important</b>: After clicking "Cell" -> "Run all", If the buttons are not appearing please restart the kernal or refresh the page.

In [ ]:
from ipywidgets import Layout, VBox, HBox
import ipywidgets as widgets
import requests as requests
from IPython.display import clear_output, Javascript
from PIL import Image
import sys
from m3inference import M3Inference
from io import BytesIO
import json
import pprint
import urllib.request
import os
from IPython.display import HTML
from pathlib import Path
import shutil
import csv
import random
import operator
import pandas as pd
import string

import warnings
warnings.filterwarnings('ignore') #hide warnings

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL) #hide info messages

%reload_ext autoreload
%autoreload 2
display(Javascript('IPython.notebook.execute_cells_below()'))

source = "../Gender_classification_M3/Trained Model/full_model.mdl"

# Destination path
destination = "/home/jovyan/m3/models/full_model.mdl"
if Path("/home/jovyan/").exists():
    if not Path("/home/jovyan/m3/models/").exists():
        os.mkdir("/home/jovyan/m3/")
        os.mkdir("/home/jovyan/m3/models/")
        dest = shutil.copyfile(source, destination)
m3 = M3Inference()

<IPython.core.display.Javascript object>

In [ ]:
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
# Running the M3 inference and getting the gender
def M3_inference():
    with open('data.jsonl') as json_file:
        data = json.load(json_file)
    pred = m3.infer(data['images'])
    with open('output.csv', 'w', newline='') as output:   # output file is created
        wr = csv.writer(output, quoting=csv.QUOTE_ALL)
        wr.writerow(['Filename', 'M3_Gender', 'Confidence'])
        for tup in pred.items():
            gender_conf = tup[1]['gender']
            gender = max(gender_conf.items(), key=operator.itemgetter(1))
            wr.writerow([tup[0], gender[0], gender[1]])
    output_df = pd.read_csv("output.csv")
    text = HTML('<p>The results are saved in the <a href="output.csv" target="_blank">csv file</a> and the folder with photos is <a href="images" target="_blank">here</a> (in case you did not press on "Clear" yet, which deletes the stored data). </p>')
    display(output_df, text, clear)

Please upload images from your computer all at once. The *name field* is optional, but M3 with both visual and textual inputs provides more accurate results. <br/>
If you want M3 to use names, select the corresponding option. The service will use filenames as firstnames for each image.


In [ ]:
# Set up layout for upload widgets
def on_upload_change(b):
    upload._counter = len(list(upload.value.keys()))

def initialize_widgets():
    global upload, name, layout
    upload = widgets.FileUpload(
        accept='image/*',
        multiple=True
    )
    name = widgets.RadioButtons(
        options=['Without names', 'With names'],
        value='Without names',
        description='M3 option:',
        disabled=False
    )
    layout = VBox([upload, name, submit])
    display(layout)
    upload.observe(on_upload_change, names='_counter')

global clear, submit, submitted, bool_names
submitted = False
bool_names = False
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

# Clear the output and delete the files


def on_clear_clicked(b):
    global submitted
    clear_output()
    initialize_widgets()
    try:
        shutil.rmtree(sys.path[0]+'/images/')
    except:
        pass
    for filename in os.listdir(sys.path[0]+'/'):
        if filename.endswith(".csv") or filename.endswith(".jsonl"):
            os.remove(filename)
    submitted = False

# Resize and convert images for M3. Save the file (is deleted afterwards)


def preprocess_and_save(content, filename):
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    image = image.resize((224, 224))
    image.save(f"images/{filename}")

# Submitting an image either as fileupload or from url


def on_submit_clicked(b):
    global submitted, content, bool_names
    if not upload.value:   # no input is given
        print('No image has been uploaded')
        return False
    if name.value == 'With names':
        bool_names = True
    else:
        bool_names = False
    display(HTML('<b>Please be patient, don\'t restart the kernel or refresh the page. The model may take some time to predict the gender.\n \n</b>'))
    file_list = list(upload.value.keys())
    if len(file_list) > 200:
        file_list = file_list[:200]
        display(HTML('The model will return the predictions only for the first 200 images.\n'))
    Path("images/").mkdir(parents=True, exist_ok=True)
    data = {}
    data['images'] = []
    for filename in file_list:
        content = upload.value[filename]['content']   # extracting the bytes for images
        preprocess_and_save(content, filename)
        if bool_names == True:
            data['images'].append({
                "description": "",
                "id": filename.split(".")[0],
                "img_path": f"images/{filename}",
                "lang": "en",
                "name": filename.split(".")[0],
                "screen_name": ""
            })
        else:
            data['images'].append({
                "description": "",
                "id": filename,
                "img_path": f"images/{filename}",
                "lang": "en",
                "name": "",
                "screen_name": ""
            })
    with open('data.jsonl', 'w') as json_file:   # json file for m3 is created
        json.dump(data, json_file)
    if submitted:   # cleaning the previous results
        clear_output()
        initialize_widgets()
    M3_inference()
    submitted = True

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)